In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc, f1_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import os
seed = 45
#seed =145
% matplotlib inline

In [2]:
PATH='F:/AV/WNS'
OOF_PATH = 'F:/AV/WNS/oof'
submission_path = 'F:/AV/WNS/submission'
stack_submission_path = 'F:/AV/WNS/submission/stack'
train_csv = 'train_catboost.csv'
submit_csv = 'sample_submission_M0L0uXE.csv'

In [3]:
oof_files = os.listdir(OOF_PATH)
removed = [i for i in oof_files if "stack" in i]
removed = removed+[i for i in oof_files if "Stack" in i]

oof_files = [i for i in oof_files if i not in removed]

oof_dict = {}
for file in oof_files:
    print(file)
    _ = pd.read_csv(f'{OOF_PATH}/{file}')
    _.columns[0]
    val = (float(_.columns[0][-5:]))
    oof_dict[file] = val

catboost_10fld_20180916082333.csv
catboost_10fld_bagtemp_20180916173347.csv
catboost_10fld_depth10_l2_1.1_temp0.1_20180916111008.csv
catboost_10fld_depth10_l2_1.1_temp1.2_20180916094559.csv
catboost_10fld_depth10_temp0.2_20180916111110.csv
catboost_10fld_depth10_temp0.3_20180916111032.csv
catboost_10fld_depth10_temp0.9_20180916094745.csv
catboost_10fld_depth10_temp1.1_20180916094605.csv
catboost_10fld_depth11_l2_1.1_temp2_20180916095129.csv
catboost_10fld_depth12_l2_1.75_temp0.5_20180916120826.csv
catboost_10fld_depth8_l2_1.1_temp0.15_20180916115751.csv
catboost_10fld_depth8_l2_1.1_temp0.5_20180916124013.csv
catboost_10fld_depth8_l2_1.1_temp3_20180916133324.csv
catboost_10fld_depth8_temp0.2_20180916120941.csv
catboost_10fld_depth8_temp0.3_20180916120820.csv
catboost_10fld_depth9_l2_1.1_temp0.55_20180916120924.csv
catboost_10fld_depth9__seed_temp0.2_20180916115740.csv
catboost_10fld_depth_10_temp0.9_20180916124610.csv
catboost_10fld_depth_10_temp5.0_20180916133913.csv
catboost_10fld_dep

In [4]:
#sorted(oof_dict.iteritems(), key=lambda (k,v): (v,k))
import operator
sorted_x = sorted(oof_dict.items(), key=operator.itemgetter(1))

In [5]:
ix = [(i[1])>0.52 for i in sorted_x]

new_dict = {}
for i in range(len(sorted_x)):
    if ix[i]:
        k,v = sorted_x[i]
        new_dict[k] = v

In [6]:
new_dict

{'lightgbm_5fld_diffseed_20180915125623.csv': 0.5201,
 'catboost_5fld_depth8_temp0.3_20180916124602.csv': 0.5209,
 'xgb_10fld_20180915183532.csv': 0.5211,
 'xgb_10fld_20180915194445.csv': 0.5215,
 'lightgbm_10fld_20180915124531.csv': 0.5234,
 'catboost_5fld_bagtemp_20180916192102.csv': 0.5236,
 'catboost_5fld_20180916025522.csv': 0.524,
 'lightgbm_10fld_20180915124747.csv': 0.5254,
 'catboost_5fld_baggingtemp0.85_20180916070735.csv': 0.5275,
 'catboost_10fld_bagtemp_20180916173347.csv': 0.5282,
 'catboost_10fld_20180916082333.csv': 0.531}

In [7]:
# oof_files = ['catboost_10fld_bagtemp_20180916173347.csv','catboost_10fld_20180916082333.csv',
#              'catboost_5fld_baggingtemp0.85_20180916070735.csv',
#             'xgb_10fld_20180915194445.csv','xgb_10fld_20180915183532.csv',
#              'lightgbm_10fld_20180915124747.csv',
#              'logreg_20180915130548.csv'
#             ]
oof_files = [i for i,j in new_dict.items()]
oof_files
test_files = oof_files
Y_true = pd.read_csv(f'{PATH}/{train_csv}')['is_promoted']
submission = pd.read_csv(f'{PATH}/{submit_csv}')

In [8]:
oof_df = pd.concat([pd.read_csv(f'{OOF_PATH}/{i}') for i in oof_files],axis=1)
test_df = pd.concat([pd.read_csv(f'{submission_path}/{i}').iloc[:,1:] for i in oof_files],axis=1)

In [9]:
Y_true.shape[0] == oof_df.shape[0]

True

In [10]:
Y_true.shape

(54808,)

In [11]:
test_df.shape

(23490, 11)

In [12]:
oof_df.shape

(54808, 11)

In [13]:
oof_df.head()

,lightgbm_5fld_diffseed_0.5201,catboost_5fld_depth8_temp0.3_0.5209,xgb_10fld_0.5211,xgb_10fld_0.5215,lightgbm_10fld_0.5234,catboost_5fld_bagtemp_0.5236,catboost_5fld_0.524,lightgbm_10fld_0.5254,catboost_5fld_baggingtemp0.85_0.5275,catboost_10fld_bagtemp_0.5282,catboost_10fld_0.531
0,0.251151,0.178219,0.276706,0.240680,0.224476,0.201242,0.211186,0.251634,0.197621,0.229450,0.233136
1,0.004188,0.034839,0.001657,0.001845,0.004827,0.017072,0.018810,0.004689,0.019009,0.015912,0.020193
2,0.000457,0.001652,0.007921,0.000501,0.000857,0.000106,0.000023,0.001129,0.000200,0.000013,0.000096
3,0.001732,0.002340,0.000353,0.000347,0.000144,0.000014,0.000157,0.000200,0.000035,0.000012,0.000031
4,0.005100,0.009518,0.002578,0.005591,0.003859,0.002555,0.003368,0.002808,0.001812,0.005401,0.002351


In [14]:
test_df.columns = oof_df.columns
test_df.head()

,lightgbm_5fld_diffseed_0.5201,catboost_5fld_depth8_temp0.3_0.5209,xgb_10fld_0.5211,xgb_10fld_0.5215,lightgbm_10fld_0.5234,catboost_5fld_bagtemp_0.5236,catboost_5fld_0.524,lightgbm_10fld_0.5254,catboost_5fld_baggingtemp0.85_0.5275,catboost_10fld_bagtemp_0.5282,catboost_10fld_0.531
0,0.186231,0.122996,0.194406,0.192050,0.196020,0.123082,0.133610,0.190533,0.139478,0.126459,0.137725
1,0.002037,0.002973,0.001497,0.000745,0.001239,0.000332,0.000554,0.001850,0.000424,0.000322,0.000372
2,0.001001,0.001510,0.001258,0.000366,0.000515,0.000015,0.000042,0.000880,0.000056,0.000022,0.000029
3,0.003680,0.004875,0.002160,0.001621,0.002301,0.000757,0.001068,0.002967,0.001002,0.000764,0.000878
4,0.002656,0.004373,0.001753,0.001443,0.001783,0.000801,0.001040,0.002334,0.000984,0.000743,0.000848


In [15]:
def kfold_logreg(train_df,test_df, target,num_folds, optimize=False,stratified = False, debug= False,modelname="lightgbm_0"):
    # Divide in training/validation and test data
    train_df = train_df
    test_df = test_df
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['employee_id','index','logreg_0.3322']]
    print(feats)

    #Grid Search
    if optimize:
        logreg = LogisticRegression()
        #param = {'C':[0.001,0.01,0.1,1,10,100]}
        param = {'C':[5,7.5,10,12.5,15,30]}
        clf = GridSearchCV(logreg,param,scoring='f1',refit=True,cv=num_folds,verbose = 1)
        clf.fit(train_df[feats],target)
        best_c = clf.best_params_['C']
        print('Best F1: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_))
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], target)):
        train_x, train_y = train_df[feats].iloc[train_idx], target[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], target[valid_idx]

        if optimize:
            clf = LogisticRegression(C=best_c)
        else:
            clf = LogisticRegression()

        clf.fit(train_x,train_y)

        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits

        print('Fold %2d F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.3).astype(int))))
        print('Fold %2d > 0.2 F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.2).astype(int))))
        print('Fold %2d > 0.1 F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.1).astype(int))))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
        
    print('Full > 0.28 f1 score %.6f' % f1_score(target, (oof_preds>0.28).astype(int)))
    print('Full > 0.30 f1 score %.6f' % f1_score(target, (oof_preds>0.29).astype(int)))
    print('Full > 0.32 f1 score %.6f' % f1_score(target, (oof_preds>0.30).astype(int)))
    print('Full > 0.25 f1 score %.6f' % f1_score(target, (oof_preds>0.31).astype(int)))
    print('Full > 0.275 f1 score %.6f' % f1_score(target, (oof_preds>0.32).astype(int)))

    # Write submission file and plot feature importance
    if not debug:
        _ = datetime.now().strftime('%Y%m%d%H%M%S')
        Fname = 'F:/AV/WNS/submission/stack/'+str(modelname)+'_'+str(_)+'.csv'
        submission['is_promoted'] = sub_preds
        submission[['employee_id', 'is_promoted']].to_csv(Fname, index= False)
        oof = pd.DataFrame(oof_preds)
        score = f1_score(target, (oof_preds>0.3).astype(int))
        oof.columns = [modelname+'_'+str(round(score,4))]
        OOF_Fname = 'F:/AV/WNS/oof/stack/'+str(modelname)+'_'+str(_)+'.csv'
        oof.to_csv(OOF_Fname,index=False)
    #display_importances(feature_importance_df)
    return 

In [18]:
import gc
oof = kfold_logreg(oof_df,test_df, Y_true,num_folds=4, optimize=False,stratified = True, debug= False,modelname="Logreg_Stack_Final_v1")

Starting LightGBM. Train shape: (54808, 11), test shape: (23490, 11)
['lightgbm_5fld_diffseed_0.5201', 'catboost_5fld_depth8_temp0.3_0.5209', 'xgb_10fld_0.5211', 'xgb_10fld_0.5215', 'lightgbm_10fld_0.5234', 'catboost_5fld_bagtemp_0.5236', 'catboost_5fld_0.524', 'lightgbm_10fld_0.5254', 'catboost_5fld_baggingtemp0.85_0.5275', 'catboost_10fld_bagtemp_0.5282', 'catboost_10fld_0.531']
Fold  1 F-score : 0.522063
Fold  1 > 0.2 F-score : 0.516538
Fold  1 > 0.1 F-score : 0.457898
Fold  2 F-score : 0.531755
Fold  2 > 0.2 F-score : 0.524942
Fold  2 > 0.1 F-score : 0.469633
Fold  3 F-score : 0.546046
Fold  3 > 0.2 F-score : 0.539315
Fold  3 > 0.1 F-score : 0.473640
Fold  4 F-score : 0.516027
Fold  4 > 0.2 F-score : 0.514183
Fold  4 > 0.1 F-score : 0.458251
Full > 0.28 f1 score 0.528519
Full > 0.30 f1 score 0.528875
Full > 0.32 f1 score 0.528846
Full > 0.25 f1 score 0.527748
Full > 0.275 f1 score 0.527948


In [17]:
#0.529741